This Python Notebook illustrates simple data analysis using USGS streamflow data that the user uploads, python code that the user uploads, and SNOTEL data that the code downloads from the NRCS. This Notebook is intended to be used as a supplement to the Hydrolearn module "Embedding data and python code into a module" HydroLearn EmbedCode101 developed by David Tarboton, 6/28/20 as part of a HydroLearn hackathon.

The cell below references dependencies that this code uses.  Depending on where you are running this notebook you may need to install these dependencies by uncommenting (removing the # from the pip install lines below)

In [ ]:
# Dependencies
!pip install numpy
!pip install matplotlib
!pip install pandas
import numpy as np
import matplotlib.pyplot as plt  # plotting

### Data and Code
The following files are provided
- FlowFromSnow.ipynb.  This notebook file
- usgs10109000.txt.  A streamflow data file from https://waterdata.usgs.gov/nwis/. Users who want to use this code with different data should download their own data from NWIS. 
- GetDataFunctions.py.  A python file that serves to illustrate user supplied code. It is python code for reading the NWIS file just read above, as well as fetching SNOTEL data from a USDA server. If you want to work with your own python code, to do your own analysis, or retrieve different data, this serves as an example.


The cell below uses the code in GetDataFunctions.py to get the data you need to work with by calling two functions 
- read_usgs_streamflow_file
- read_snotel
Results are returned in Pandas data frames.  

In [ ]:
# Read the data using functions in GetDataFunctions.py
from GetDataFunctions import *
flowdata=read_usgs_streamflow_file("usgs10109000.txt")
flowdata.plot(x='date',y='flow')

snow=read_snotel(823,"UT","2020-06-20")  #enddt="2020-06-20"
snow.plot(x='date',y='swe')

The next cell does some simple analysis. A hypothesis is that the maximum snow water equivalent at a SNOTEL site that occurs on about April 1 of each year is a reasonable predictor of the maximum daily streamflow that may occur around June in a stream driven by snowmelt.

For the example data provided, Logan River, near Logan Utah, and Tony Grove Lake SNOTEL site you will see that the Hypothesis is reasonable.

The code below scans each complete year of SNOTEL data to determine the maximum snow water equivalent. It scans daily streamflow data to determine the maximum daily streamflow. It then plots max streamflow vs max snow, and draws a best fit straight line, showing the equation that could be fit.

There is a lot more that could be done with this example, but it stops here, as the purpose is to keep this short and simple as an illustration of how to work with data embedded into HydroLearn

In [ ]:
# Analysis to develop simple linear model for Maximum daily streamflow as a function of Maximum snow water equivalent
ybeg=int(snow.head(1)['date'].dt.year)+1
yend=int(snow.tail(1)['date'].dt.year)-1
yrs=[]
snowmax=[]
flowmax=[]
for yr in range(ybeg,yend):
    yrs.append(yr)
    start_date=str(yr)+'-01-01'
    end_date=str(yr)+'-12-31'
    snowmax.append(max(snow.loc[(snow['date'] >= start_date) & (snow['date'] <= end_date)]['swe']))
    flowmax.append(max(flowdata.loc[(flowdata['date'] >= start_date) & (flowdata['date'] <= end_date)]['flow']))
snowmax=np.array(snowmax)
flowmax=np.array(flowmax)
plt.scatter(snowmax,flowmax)
plt.xlabel('Max SWE (inches)')
plt.ylabel('Max Discharge (cfs)')
# Fit a straight line using numpy's polyfit function
m,b=np.polyfit(snowmax,flowmax,1)
plt.plot(snowmax, m*snowmax+b)
print("Regression fit:")
print("Max daily streamflow = "+str(round(b,3))+" + "+str(round(m,3))+"*(Max snow water equivalent in inches)  (cfs)")
